# Knet Neural Networks
***
* Objective: Learning construction a model like the LeNet given in the [Quick Start](https://github.com/denizyuret/Knet.jl/blob/master/tutorial/15.quickstart.ipynb) with a thorough explanation of each part
* Prerequisites: [Julia arrays](https://docs.julialang.org/en/v1/manual/arrays)

In Knet, there are two ways to create a neural network, one being using the built-in Knet structs and the other being defining hand-written callable objects that accepts an array (matrix or vector) and outputs the result of the wanted logic in the right dimension. We will begin with custom layer.

Importing Knet if not already imported or using it

In [ ]:
#using Pkg
#Pkg.add("Knet")
using Knet
#You may see an error if your device does not support CUDA or your CUDA driver is not CUDA 10.1 or higher

## Creating Our First Custom Layer

A sample dense layer:

In [ ]:
struct dense; w; b; f; end

Definition of a sample dense layer includes w (weights), b (bias), f(activation function). Bias and activation functions are not necessary but the field "w" is needed in all custom layers to be manipulated during training process.

In [ ]:
(d::dense)(x) = d.f.(d.w * mat(x) .+ d.b)

We create a functor (a callable object/ function-like object) that multiplies the input with its weights, adds bias, applies the given activation function and returns the results

In [ ]:
dense(i::Int,o::Int,f=relu) = dense(param(o,i), param0(o), f);

We declare a constructor that utilizes built-in "param" and "param0" functions that return KnetArrays, powerful built-in array that have all the operations a Julia array has but also designed with a focus on GPU opearations, in the appropriate sizes. Usage of Knet arrays are not mandatory however highly encouraged due to performance improvement. 

In addition, the constructor has a keyword-parameter named "f", whose default value is relu. Relu is one of the many built-in activation functions Knet has. Here is a list of all the built-in Knet functions: elu, relu, selu, sigmoid, gelu.
For further reference:

In [ ]:
@doc relu

In [ ]:
@doc elu

That is how custom layers are defined in Knet

Here is another example custom layer:.

In [ ]:
struct Conv; w; b; f; end
(c::Conv)(x) = c.f.(pool(conv4(c.w, x) .+ c.b))
Conv(w1,w2,cx,cy,f=relu) = Conv(param(w1,w2,cx,cy), param0(1,1,cy,1), f);

As mentioned before, a struct with the fields including  the "w" field representing the weights has been declared. Then, a function-like object that takes an input (matrix, number or vector) and outputs the result of the appliance of the inner logic. This struct makes use of the param and param0 as well. Since this is a convolutional layer, the calculation has different steps. For further reference:

In [ ]:
 @doc conv4

In [ ]:
@doc pool

## Using built-in Layers
(To include this part, the [layers script](https://github.com/denizyuret/Knet.jl/blob/master/src/layers21/Layers21.jl) must export the layers
Knet offers the following built-in layers with a remarkable option of customization:
* [Dense](https://github.com/denizyuret/Knet.jl/blob/master/src/layers21/dense.jl)
* [Embed](https://github.com/denizyuret/Knet.jl/blob/master/src/layers21/embed.jl)

### Dense Layer 
Built-in Dense Layer has the following constructors:

* function Dense(weights, bias=nothing; f=nothing, dims=1, dropout=0)
* function Dense(wsize::Integer...; f=nothing, dims=1, dropout=0, atype=atype(), binit=zeros,  init=𝑼(√(6/(densein(wsize,dims)+denseout(wsize,dims)))))

Although how confusion the definitions may seem at the first glance, they are fairly easy to use. 

Keyword arguments:
* `f=nothing`: apply activation function to output if not nothing
* `dims=1`: number of input dimensions in the weight tensor
* `dropout=0`: apply dropout with this probability to input if non-zero
* `atype=Knet.atype()`: array and element type for parameter initialization
* `init=𝑼(√(6/(fanin+fanout)))`: initialization function for weights
* `binit=zeros`: initialization function for bias, if `nothing` do not use bias

Example:

In [ ]:
dense_layer = Dense(2, dim = 2, f = relu)

## Chaining Layers
To chain layers, one must declare a callable object that can iterate over the layers and output the final value or return the result of the cost function when (x,y) provided. Currently, Knet does not have a built-in model that is capable of these operations. An example is placed below:

In [ ]:
struct Chain; layers; Chain(args...)= new(args);end

A simple struct that has a field layers which will hold the given layers as a tuple. 

In [ ]:
(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)

A callable object is created that takes an input and applies the logic given in the layers in the given order

In [ ]:
(c::Chain)(x,y) = nll(c(x),y)

A callable object that returns the cost of a(x) and y (true) 

## Creating the model

In [ ]:
LeNet = Chain(Conv(5,5,1,20), Conv(5,5,20,50), dense(800,500), dense(500,10,identity))